In [1]:
import matplotlib as mpl
mpl.use('pgf')

mpl.rcParams.update({
    'pgf.texsystem': 'pdflatex',
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,  
})

import matplotlib.pyplot as plt
# Show the image with ipywidges
import matplotlib.dates as mdates

import pandas as pd
import datetime
import json
import numpy as np
import seaborn as sns

In [2]:
with open('data/processed_things_2024_01_22.json') as f:
    processed_things = json.load(f)

distances_week = [[] for _ in range(7 * 24)]
noises_week = [[] for _ in range(7 * 24)]
green_lengths_week = [[] for _ in range(7 * 24)]
green_phase_count_week = [[] for _ in range(7 * 24)]

for thing_name, thing in processed_things.items():
    if thing["TotalCyclesCount"] == 0:
        continue
    if thing["TotalRemovedCycleCount"] / thing["TotalCyclesCount"] > 0.1:
        continue
    for day_idx in range(7):
        for hour_idx in range(24):
            if thing["Metrics"][day_idx][hour_idx] != -1.0:
                d = thing["Metrics"][day_idx][hour_idx]
                distances_week[day_idx * 24 + hour_idx].append(d)
            if thing["ShiftsFuzzyness"][day_idx][hour_idx] != -1.0:
                n = thing["ShiftsFuzzyness"][day_idx][hour_idx]
                noises_week[day_idx * 24 + hour_idx].append(n)
            if thing["MedianGreenLengths"][day_idx][hour_idx] != -1.0:
                g = thing["MedianGreenLengths"][day_idx][hour_idx]
                green_lengths_week[day_idx * 24 + hour_idx].append(g)
            if thing['GreenPhaseCount'][day_idx][hour_idx] != -1.0:
                g = thing['GreenPhaseCount'][day_idx][hour_idx]
                green_phase_count_week[day_idx * 24 + hour_idx].append(g)
                
# Convert to local time zone (shift two hours)
distances_week = distances_week[-2:] + distances_week[:-2]
noises_week = noises_week[-2:] + noises_week[:-2]
green_lengths_week = green_lengths_week[-2:] + green_lengths_week[:-2]
green_phase_count_week = green_phase_count_week[-2:] + green_phase_count_week[:-2]

In [3]:
observations_week_car = [0 for _ in range(7 * 24)]
observations_week_cyclists = [0 for _ in range(7 * 24)]
observations_week_pedestrian = [0 for _ in range(7 * 24)]
observations_week_bus_checkout = [0 for _ in range(7 * 24)]
observations_week_bus_request = [0 for _ in range(7 * 24)]

with open('data/observations_per_hour/observations_per_hour_timestamp_counts_detector_car.json') as f:
    observations_per_hour_timestamp_counts_detector_car = json.load(f)
with open('data/observations_per_hour/observations_per_hour_timestamp_counts_detector_cyclists.json') as f:
    observations_per_hour_timestamp_counts_detector_cyclists = json.load(f)
with open('data/observations_per_hour/observations_per_hour_timestamp_counts_detector_pedestrian.json') as f:
    observations_per_hour_timestamp_counts_detector_pedestrian = json.load(f)
with open('data/observations_per_hour/observations_per_hour_timestamp_counts_bus_checkout.json') as f:
    observations_per_hour_timestamp_counts_detector_bus_checkout = json.load(f)
with open('data/observations_per_hour/observations_per_hour_timestamp_counts_bus_request_point.json') as f:
    observations_per_hour_timestamp_counts_detector_bus_request_point = json.load(f)

for timestamp, count in observations_per_hour_timestamp_counts_detector_car.items():
    int_timestamp = int(timestamp.replace(".0", ""))
    if int_timestamp >= 1697752800:
        continue
    if int_timestamp < 1695333600:
        continue
    dt = datetime.datetime.fromtimestamp(float(timestamp))
    observations_week_car[dt.weekday() * 24 + dt.hour] += count
for timestamp, count in observations_per_hour_timestamp_counts_detector_cyclists.items():
    int_timestamp = int(timestamp.replace(".0", ""))
    if int_timestamp >= 1697752800:
        continue
    if int_timestamp < 1695333600:
        continue
    dt = datetime.datetime.fromtimestamp(float(timestamp))
    observations_week_cyclists[dt.weekday() * 24 + dt.hour] += count
for timestamp, count in observations_per_hour_timestamp_counts_detector_pedestrian.items():
    int_timestamp = int(timestamp.replace(".0", ""))
    if int_timestamp >= 1697752800:
        continue
    if int_timestamp < 1695333600:
        continue
    dt = datetime.datetime.fromtimestamp(float(timestamp))
    observations_week_pedestrian[dt.weekday() * 24 + dt.hour] += count
for timestamp, count in observations_per_hour_timestamp_counts_detector_bus_checkout.items():
    int_timestamp = int(timestamp.replace(".0", ""))
    if int_timestamp >= 1697752800:
        continue
    if int_timestamp < 1695333600:
        continue
    dt = datetime.datetime.fromtimestamp(float(timestamp))
    observations_week_bus_checkout[dt.weekday() * 24 + dt.hour] += count
for timestamp, count in observations_per_hour_timestamp_counts_detector_bus_request_point.items():
    int_timestamp = int(timestamp.replace(".0", ""))
    if int_timestamp >= 1697752800:
        continue
    if int_timestamp < 1695333600:
        continue
    dt = datetime.datetime.fromtimestamp(float(timestamp))
    observations_week_bus_request[dt.weekday() * 24 + dt.hour] += count

# Move the last 24 hours to the start, to begin with sunday
observations_week_car = observations_week_car[-24:] + observations_week_car[:144]
observations_week_cyclists = observations_week_cyclists[-24:] + observations_week_cyclists[:144]
observations_week_pedestrian = observations_week_pedestrian[-24:] + observations_week_pedestrian[:144]
observations_week_bus_checkout = observations_week_bus_checkout[-24:] + observations_week_bus_checkout[:144]
observations_week_bus_request = observations_week_bus_request[-24:] + observations_week_bus_request[:144]

# Normalize
# observations_week_car = [x / max(observations_week_car) for x in observations_week_car]
# observations_week_cyclists = [x / max(observations_week_cyclists) for x in observations_week_cyclists]
# observations_week_pedestrian = [x / max(observations_week_pedestrian) for x in observations_week_pedestrian]
# observations_week_bus_checkout = [x / max(observations_week_bus_checkout) for x in observations_week_bus_checkout]
# observations_week_bus_request = [x / max(observations_week_bus_request) for x in observations_week_bus_request]

# Convert to local time zone (shift two hours)
observations_week_car = observations_week_car[-2:] + observations_week_car[:166]
observations_week_cyclists = observations_week_cyclists[-2:] + observations_week_cyclists[:166]
observations_week_pedestrian = observations_week_pedestrian[-2:] + observations_week_pedestrian[:166]
observations_week_bus_checkout = observations_week_bus_checkout[-2:] + observations_week_bus_checkout[:166]
observations_week_bus_request = observations_week_bus_request[-2:] + observations_week_bus_request[:166]

total_car_observations = sum(observations_week_car)
total_cyclists_observations = sum(observations_week_cyclists)
total_pedestrian_observations = sum(observations_week_pedestrian)
total_bus_checkout_observations = sum(observations_week_bus_checkout)
total_bus_request_observations = sum(observations_week_bus_request)

print("Total car observations: {}".format(total_car_observations))
print("Total cyclist observations: {}".format(total_cyclists_observations))
print("Total pedestrian observations: {}".format(total_pedestrian_observations))
print("Total bus checkout observations: {}".format(total_bus_checkout_observations))
print("Total bus request observations: {}".format(total_bus_request_observations))

total_detector_observations = total_car_observations + total_cyclists_observations + total_pedestrian_observations + total_bus_request_observations

print("Total detector observations: {}".format(total_detector_observations))

Total car observations: 612179842
Total cyclist observations: 13338474
Total pedestrian observations: 13912603
Total bus checkout observations: 2185312
Total bus request observations: 2223354
Total detector observations: 641654273


In [15]:
from matplotlib.colors import LogNorm

green_lengths_hist = []
for hour in green_lengths_week:
    green_lengths_hist.append(np.histogram(hour, bins=100, range=(0, 100))[0][::-1])

# Make a histogram for each hour
distances_week_hists = []
distances_week_medians = []
for hour in distances_week:
    distances_week_hists.append(np.histogram(hour, bins=50, range=(0, 50))[0][::-1])
    distances_week_medians.append(np.median(hour))

noises_week_hists = []
noises_week_median = []
for hour in noises_week:
    noises_week_hists.append(np.histogram(hour, bins=50, range=(0, 1))[0][::-1])
    noises_week_median.append(np.median(hour))

# Get vmax and vmin
vmin = 1
vmax = 0
for hour in distances_week_hists:
    for value in hour:
        if value > vmax:
            vmax = value
for hour in noises_week_hists:
    for value in hour:
        if value > vmax:
            vmax = value
for hour in green_lengths_hist:
    for value in hour:
        if value > vmax:
            vmax = value

print("vmax", vmax)
print("vmin", vmin)

# Plot heatmap
fig, ((ax0), (ax1), (ax3), (ax9), (ax7), (ax5)) = plt.subplots(figsize=(5, 9), nrows=6, ncols=1, gridspec_kw={'height_ratios': [0.05, 1, 1, 1, 1, 1]})
im = ax1.imshow(np.array([x for x in distances_week_hists]).T, cmap='jet', interpolation='none', aspect='auto', norm=LogNorm(vmin=vmin, vmax=vmax, clip=True), rasterized=True)

# ax0 is the colorbar
cbar = fig.colorbar(im, cax=ax0, orientation='horizontal')
cbar.ax.xaxis.set_ticks_position('top')
cbar.ax.xaxis.set_tick_params(labelsize=8)
cbar.ax.xaxis.set_label_position('top')
cbar.ax.set_xlabel('\# Hourly Buckets', fontsize=10)

# Add labels
ax1.set_xticks(range(0, 24))
hours = range(0, 24 * 7, 12)
hours_human_readable = []
for h in hours:
    hours_human_readable.append(datetime.time(h % 24).strftime("%H"))
ax1.set_xticks(hours)
ax1.set_xticklabels([])
ax1.set_xlim(0, 24 * 7 - 1)
ax1.set_yticks(range(0, 51, 10))
ax1.set_yticklabels([f"{x}s" for x in range(0, 51, 10)][::-1], fontsize=8)
ax1.set_title("Cycle Discrepancy", fontsize=10)

for i in range(1, 14):
    ax1.axvline(x=i * 12, color='black', linestyle='-', alpha=0.05, linewidth=1)

ax1.set_facecolor('black')

# Plot heatmap
im = ax3.imshow(np.array([x for x in noises_week_hists]).T, cmap='jet', interpolation='none', aspect='auto', norm=LogNorm(vmin=vmin, vmax=vmax, clip=True), rasterized=True)

# Add labels
ax3.set_xticks(range(0, 24))
hours = range(0, 24 * 7, 12)
hours_human_readable = []
for h in hours:
    hours_human_readable.append(datetime.time(h % 24).strftime("%H"))
ax3.set_xticks(hours)
ax3.set_xticklabels([])

ax3.set_yticks([0, 12.5, 25, 37.5, 50])
ax3.set_yticklabels(["100%", "75%", "50%", "25%", "0%"], fontsize=8)
ax3.set_title("Wait Time Diversity", fontsize=10)
for i in range(1, 14):
    ax3.axvline(x=i * 12, color='black', linestyle='-', alpha=0.05, linewidth=1)

ax3.set_facecolor('black')

# Plot heatmap
im = ax9.imshow(np.array([x for x in green_lengths_hist]).T, cmap='jet', interpolation='none', aspect='auto', norm=LogNorm(vmin=vmin, vmax=vmax, clip=True), rasterized=True)

# Add labels
ax9.set_xticks(range(0, 24))
hours = range(12, 24 * 7, 24)
hours_human_readable = []
for h in hours:
    hours_human_readable.append(datetime.time(h % 24).strftime("%H:%M"))
ax9.set_xticks(hours)
ax9.set_xticklabels([])

ax9.set_yticks([0, 25, 50, 75, 100])
ax9.set_yticklabels(["100s", "75s", "50s", "25s", "0s"], fontsize=8)
ax9.set_title("Green Length", fontsize=10)

for i in range(1, 14):
    ax9.axvline(x=i * 12, color='black', linestyle='-', alpha=0.05, linewidth=1)

ax9.set_facecolor('black')

# Make a histogram for each hour
total_green_phases_counts_week = []
for hour in green_phase_count_week:
    count = 0
    for green_phase_count in hour:
        count += green_phase_count
    total_green_phases_counts_week.append(count)

ax7.set_title("\# Green Phases", fontsize=10)
ax7.step(range(0, 24 * 7), total_green_phases_counts_week,  color=sns.color_palette('Set2')[4], linewidth=1)
# Paint below
ax7.fill_between(range(0, 24 * 7), total_green_phases_counts_week, color=sns.color_palette('Set2')[4], alpha=0.2)
ax7.set_xlim(0, 24 * 7 - 1)
ax7.set_ylim(0, 3_000_000)
ax7.set_yticks([0, 1_000_000, 2_000_000, 3_000_000])
ax7.set_yticklabels(["0", "1M", "2M", "3M"], fontsize=8)
ax7.set_xticks(hours)
ax7.set_xticklabels([])
for i in range(1, 14):
    ax7.axvline(x=i * 12, color='grey', linestyle='-', alpha=0.1, linewidth=1)

ax5.set_title("\# Detector Occupancy Changes", fontsize=10)
summed = [x + y + z + w + v for x, y, z, w, v in zip(observations_week_car, observations_week_cyclists, observations_week_pedestrian, observations_week_bus_checkout, observations_week_bus_request)]
ax5.step(range(0, 24 * 7), summed, color=sns.color_palette('Set2')[0], linewidth=1)
# Paint below
ax5.fill_between(range(0, 24 * 7), summed, color=sns.color_palette('Set2')[0], alpha=0.2)
ax5.set_ylim(0, 8_000_000)
ax5.set_yticks([0, 2_000_000, 4_000_000, 6_000_000, 8_000_000])
ax5.set_yticklabels(["0", "2M", "4M", "6M", "8M"], fontsize=8)
ax5.set_xlim(0, 24 * 7 - 1)
ax5.set_xticks(hours)
ax5.set_xticklabels(hours_human_readable, fontsize=8)
# vertical line every 12 hours
for i in range(1, 14):
    ax5.axvline(x=i * 12, color='grey', linestyle='-', alpha=0.1, linewidth=1)
    
start_x = 0.18
diff_x = 0.11175
y = 0.07
x_coords = [start_x + diff_x * i for i in range(7)]

ax5.text(x_coords[0], y, "SUN", fontsize=8, ha="center", transform=plt.gcf().transFigure)
ax5.text(x_coords[1], y, "MON", fontsize=8, ha="center", transform=plt.gcf().transFigure)
ax5.text(x_coords[2], y, "TUE", fontsize=8, ha="center", transform=plt.gcf().transFigure)
ax5.text(x_coords[3], y, "WED", fontsize=8, ha="center", transform=plt.gcf().transFigure)
ax5.text(x_coords[4], y, "THU", fontsize=8, ha="center", transform=plt.gcf().transFigure)
ax5.text(x_coords[5], y, "FRI", fontsize=8, ha="center", transform=plt.gcf().transFigure)
ax5.text(x_coords[6], y, "SAT", fontsize=8, ha="center", transform=plt.gcf().transFigure)

# Set minor locator to 1 hour
for ax in [ax1, ax3, ax9, ax7, ax5]:
    from matplotlib.ticker import MultipleLocator
    ax.xaxis.set_minor_locator(MultipleLocator(6))
    ax.tick_params(which='minor', length=2)

plt.subplots_adjust(wspace=0.02, hspace=0.3)

fig_name = 'figures/predictability-week-heatmap'
plt.savefig(f'{fig_name}.pdf', bbox_inches='tight')
plt.savefig(f'{fig_name}.png', bbox_inches='tight', dpi=300)
plt.savefig(f'{fig_name}.pgf', bbox_inches='tight', dpi=300)

print("Median distance", np.median(distances_week_medians))
print("Min median distance", np.min(distances_week_medians))
print("Max median distance", np.max(distances_week_medians))
print("Median noise", np.median(noises_week_median))
print("Min median noise", np.min(noises_week_median))
print("Max median noise", np.max(noises_week_median))

vmax 9717
vmin 1
Median distance 1.0
Min median distance 0.0
Max median distance 2.0
Median noise 0.05495169082125603
Min median noise 0.04054054054054054
Max median noise 0.09243697478991597


In [6]:
from matplotlib.colors import LogNorm

# Make a histogram for each hour
noises_week_hists = []
for hour in noises_week:
    noises_week_hists.append(np.histogram(hour, bins=50, range=(0, 1))[0][::-1])
# Plot heatmap
fig, (ax1, ax3) = plt.subplots(figsize=(10, 2.5), nrows=1, ncols=2, gridspec_kw={'width_ratios': [1, 0.05]})
im = ax1.imshow(np.array([x for x in noises_week_hists]).T, cmap='jet', interpolation='none', aspect='auto', norm=LogNorm(), rasterized=True)

# Add labels
ax1.set_xticks(range(0, 24))
hours = range(0, 24 * 7, 6)
hours_human_readable = []
for h in hours:
    hours_human_readable.append(datetime.time(h % 24).strftime("%H"))
ax1.set_xticks(hours)
ax1.set_xticklabels([])

ax1.set_yticks([0, 50])
ax1.set_yticklabels([1, 0])
ax1.set_ylabel("Cycle-independent\ninstability (\%)")

ax1.text(12, -2, "Sundays", fontsize=10, ha="center")
ax1.text(24 + 12, -2, "Mondays", fontsize=10, ha="center")
ax1.text(48 + 12, -2, "Tuesdays", fontsize=10, ha="center")
ax1.text(72 + 12, -2, "Wednesdays", fontsize=10, ha="center")
ax1.text(96 + 12, -2, "Thursdays", fontsize=10, ha="center")
ax1.text(120 + 12, -2, "Fridays", fontsize=10, ha="center")
ax1.text(144 + 12, -2, "Saturdays", fontsize=10, ha="center")

# Add a colorbar, but plot it in ax3
cbar = fig.colorbar(im, cax=ax3, orientation='vertical')
cbar.ax.set_ylabel("Number of traffic lights", rotation=-90, va="bottom")

plt.subplots_adjust(wspace=0.1, hspace=0.1)

fig_name = 'figures/predictability-week-heatmap-cycle-independent'
plt.savefig(f'{fig_name}.pdf', bbox_inches='tight')
plt.savefig(f'{fig_name}.png', bbox_inches='tight', dpi=300)

In [6]:
print(green_lengths_week[-1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [3]:
from matplotlib.colors import LogNorm

# Make a histogram for each hour
green_lengths_week_hists = []
for hour in green_lengths_week:
    green_lengths_week_hists.append(np.histogram(hour, bins=50, range=(0, 100))[0][::-1])
# Plot heatmap
fig, (ax1, ax3) = plt.subplots(figsize=(10, 2.5), nrows=1, ncols=2, gridspec_kw={'width_ratios': [1, 0.05]})
im = ax1.imshow(np.array([x for x in green_lengths_week_hists]).T, cmap='jet', interpolation='none', aspect='auto', norm=LogNorm(), rasterized=True)

# Add labels
ax1.set_xticks(range(0, 24))
hours = range(0, 24 * 7, 6)
hours_human_readable = []
for h in hours:
    hours_human_readable.append(datetime.time(h % 24).strftime("%H"))
ax1.set_xticks(hours)
ax1.set_xticklabels([])

ax1.set_yticks([0, 50])
ax1.set_yticklabels([100, 0])
ax1.set_ylabel("Median green\nlength (s)")

ax1.text(12, -2, "Sundays", fontsize=10, ha="center")
ax1.text(24 + 12, -2, "Mondays", fontsize=10, ha="center")
ax1.text(48 + 12, -2, "Tuesdays", fontsize=10, ha="center")
ax1.text(72 + 12, -2, "Wednesdays", fontsize=10, ha="center")
ax1.text(96 + 12, -2, "Thursdays", fontsize=10, ha="center")
ax1.text(120 + 12, -2, "Fridays", fontsize=10, ha="center")
ax1.text(144 + 12, -2, "Saturdays", fontsize=10, ha="center")

# Add a colorbar, but plot it in ax3
cbar = fig.colorbar(im, cax=ax3, orientation='vertical')
cbar.ax.set_ylabel("Number of traffic lights", rotation=-90, va="bottom")

plt.subplots_adjust(wspace=0.1, hspace=0.1)

fig_name = 'figures/predictability-week-heatmap-green-lengths'
plt.savefig(f'{fig_name}.pdf', bbox_inches='tight')
plt.savefig(f'{fig_name}.png', bbox_inches='tight', dpi=300)

In [46]:
with open('data/processed_things_2024_01_22.json') as f:
    processed_things = json.load(f)

distances = []
noises = []

# First, order thing by sum of distances
things_sums = {}
for thing_name, thing in processed_things.items():
    # if not "_primary" in thing_name:
    #     continue
    if thing["TotalCyclesCount"] == 0:
        continue
    if thing["TotalRemovedCycleCount"] / thing["TotalCyclesCount"] > 0.1:
        continue
    thing_sum = 0
    for day_idx in range(7):
        for hour_idx in range(24):
            if thing["Metrics"][day_idx][hour_idx] != -1.0 and thing["ShiftsFuzzyness"][day_idx][hour_idx] != -1.0:
                d = thing["Metrics"][day_idx][hour_idx]
                thing_sum += d
            else:
                thing_sum += 0

    things_sums[thing_name] = thing_sum

things_sums = {k: v for k, v in sorted(things_sums.items(), key=lambda item: item[1])}

for thing_name, _ in things_sums.items():
    thing = processed_things[thing_name]
    distances_thing = []
    noises_thing = []
    for day_idx in range(7):
        for hour_idx in range(24):
            if thing["Metrics"][day_idx][hour_idx] != -1.0 and thing["ShiftsFuzzyness"][day_idx][hour_idx] != -1.0:
                d = thing["Metrics"][day_idx][hour_idx]
                n = thing["ShiftsFuzzyness"][day_idx][hour_idx]
                g = thing["MedianGreenLengths"][day_idx][hour_idx]
                distances_thing.append(d)
                noises_thing.append(n)
            else:
                distances_thing.append(-1)
                noises_thing.append(-1)
    distances.append(distances_thing)
    noises.append(noises_thing)

In [111]:
from matplotlib.colors import LogNorm

fig, ((ax1, ax3), (ax2, ax4)) = plt.subplots(figsize=(5, 5.22), nrows=2, ncols=2, gridspec_kw={'height_ratios': [1, 1], 'width_ratios': [1, 0.03]})

im = ax1.imshow(np.array(distances).T, cmap='jet', interpolation='none', aspect='auto', rasterized=True, vmin=0, vmax=50)
hours = range(12, 24 * 7, 24)
hours_human_readable = []
days = ["SUN", "MON", "TUE", "WED", "THU", "FRI", "SAT"]
for h, day in zip(hours, days):
    hours_human_readable.append(datetime.time(h % 24).strftime(f"{day}\n%H:%M"))
ax1.set_yticks(hours)
ax1.set_yticklabels(hours_human_readable, fontsize=8)
ax1.set_xticklabels([])
cbar = fig.colorbar(im, cax=ax3, orientation='vertical')
cbar.ax.set_ylabel("Cycle Discrepancy (s)", rotation=90, va="bottom", labelpad=12)

im = ax2.imshow(np.array(noises).T, cmap='jet', interpolation='none', aspect='auto', rasterized=True, vmin=0, vmax=1)
ax2.set_yticks(hours)
ax2.set_yticklabels(hours_human_readable, fontsize=8)
ax2.set_xlabel("Traffic lights (ordered by increasing cumulative cycle discrepancy)")
cbar = fig.colorbar(im, cax=ax4, orientation='vertical')
cbar.ax.set_ylabel("Wait Time Diversity (\%)", rotation=90, va="bottom", labelpad=12)

plt.subplots_adjust(wspace=0.03, hspace=0.1)

fig_name = 'figures/predictability-week-heatmap-per-thing'
plt.savefig(f'{fig_name}.pdf', bbox_inches='tight')
plt.savefig(f'{fig_name}.png', bbox_inches='tight', dpi=300)
plt.savefig(f'{fig_name}.pgf', bbox_inches='tight', dpi=300)